# <b><center>Feature Engineering</center></b>

## Import libaries

In [1]:
# ! pip install sentence_transformers

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline
# Set the option to display all columns
pd.set_option('display.max_columns', None)

## Import Data

In [3]:
df__bk_rat = pd.read_csv(r'/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', encoding="latin-1")
df_bk_dt = pd.read_csv(r'/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX_Books.csv', sep=';', encoding="latin-1")
df_usr_dt = pd.read_csv('/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Users.csv', sep=';', encoding="latin-1")
prepo_df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [4]:
prepo_df.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


## Extract Features 

### Age Group

In [5]:
# Define the number of age bins
num_age_bins = 5  # You can adjust this number based on your preference

# Create age bins
prepo_df["age_group"] = pd.cut(prepo_df["age"], bins=num_age_bins, labels=False)

In [6]:
prepo_df['age_group'].unique()

array([0, 1, 2, 3, 4])

### Continet

In [7]:
def get_continent(country):
    # Check for NaN values
    if pd.isna(country):
        return "Other"

    # Define a mapping between continents and some keywords that might appear in country names
    continent_mapping = {
        "North America": ["usa", "canada", "mexico"],
        "Europe": ["europe", "uk", "germany", "france", "spain", "italy"],
        "Asia": ["asia", "china", "japan", "india"],
        "South America": ["south america", "brazil", "argentina"],
        "Africa": ["africa", "nigeria", "egypt", "south africa"],
        "Australia": ["australia"],
        "Antarctica": ["antarctica"]
    }

    # Check if any continent keyword is present in the country name
    country = country.lower()
    for continent, keywords in continent_mapping.items():
        for keyword in keywords:
            if keyword in country:
                return continent

    # If no continent is found, return "Other"
    return "Other"

prepo_df["continent"] = prepo_df["country"].apply(get_continent)

In [8]:
prepo_df.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country,age_group,continent
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa,0,North America
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada,1,North America
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada,2,North America
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN,1,Other
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada,1,North America


### Books categories 

In [9]:
prepo_df['Category'].nunique()

6448

we need to minimize it

In [10]:
prepo_df.groupby(['Category'])['Category'].count().sort_values()

Category
['Employee rights']                             1
['Express trains']                              1
['Extrasensory perception in animals.']         1
['Extraterrestrial anthropology']               1
['Extraterrestrial anthropology.']              1
                                            ...  
['Humor']                                    8500
['Biography & Autobiography']               22390
['Juvenile Fiction']                        39617
['Fiction']                                385135
9                                          406102
Name: Category, Length: 6448, dtype: int64

let's clean commas and puncuations 

In [11]:
prepo_df['Category'] = prepo_df['Category'].str.strip(r'[]\'.,\"')

In [12]:
prepo_df.groupby(['Category'])['Category'].count()

Category
0                                                         4
0Islands                                                  3
1. Arctic                                                 1
101 dalmatians (Motion picture)                           2
11030 - fiction in English - 1900-1945 - 60030 - texts    2
                                                         ..
Zoo animals                                               2
Zoology                                                   1
Zoos                                                      1
Zululand (South Africa)                                   1
avstrijska književnost - mladinska književnost - roman    1
Name: Category, Length: 6172, dtype: int64

we lowered the number from 6,448 to 6,172. next, lets continue to minimize. how many categories are with more than 1,000 books?

In [13]:
len(prepo_df.groupby(['Category'])['Category'].count()[prepo_df.groupby(['Category'])['Category'].count()>1000])

41

unite similar categories

In [14]:
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:11]!='Young Adult' else 'Young Adult')
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:11]!='Young adult' else 'Young Adult')
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:3]!='Zoo' else 'Zoo')
prepo_df['Category'] = prepo_df['Category'].apply(lambda txt: txt if txt[0:7]!='Cookery' else 'Cookery')
#create a a higher hierarchy for category
prepo_df['big_Category'] = prepo_df['Category'].copy()
prepo_df['big_Category'] = prepo_df['big_Category'].apply(lambda txt: 'literary' if 'literary' in txt else txt)
prepo_df['big_Category'] = prepo_df['big_Category'].apply(lambda txt: 'fiction' if 'fiction' in txt else txt)

In [15]:
prepo_df['Category'] = prepo_df['Category'].str.lower()

In [16]:
prepo_df.groupby(['Category'])['Category'].count()[prepo_df.groupby(['Category'])['Category'].count()>1000].sort_values(ascending=False)#[df.groupby(['Category'])['Category'].count()>500]

Category
9                                406102
fiction                          386538
juvenile fiction                  39891
biography & autobiography         22531
humor                              8500
history                            8456
religion                           7292
juvenile nonfiction                5481
social science                     5266
body, mind & spirit                4574
business & economics               4543
family & relationships             4446
self-help                          4326
health & fitness                   3933
cooking                            2899
travel                             2743
true crime                         2639
psychology                         2541
literary criticism                 2235
poetry                             2202
science                            2156
drama                              2090
computers                          2051
political science                  1951
nature                         

In [17]:
prepo_df.groupby(['Category'])['Category'].count()

Category
0                                                            4
0islands                                                     3
1. arctic                                                    1
101 dalmatians (motion picture)                              2
11030 - fiction in english - 1900-1945 - 60030 - texts       2
                                                            ..
zen buddhism                                                 3
zeugen jehovas - weibliche jugend - ausstieg - jugendbuch    2
zip code                                                     1
zoo                                                          4
zululand (south africa)                                      1
Name: Category, Length: 6086, dtype: int64

#### BERT to caculate categories similarity

In [18]:
# Create a SentenceTransformer model (BERT-based)
model = SentenceTransformer("bert-base-nli-mean-tokens")

In [19]:
# Encode category names into embeddings
category_embeddings = model.encode(prepo_df["Category"], show_progress_bar=True)

Batches:   0%|          | 0/32225 [00:00<?, ?it/s]

In [ ]:
# Use Agglomerative Clustering to group similar categories
clustering_model = AgglomerativeClustering(n_clusters=10)  # You can adjust the number of clusters
clusters = clustering_model.fit_predict(category_embeddings)

# Assign cluster labels to the DataFrame
prepo_df["grouped_category"] = clusters

### Save to new csv 

In [ ]:
#Save the modified DataFrame back to a CSV file if needed
data.to_csv("processed_data_upd.csv", index=False)